# Improve response parsing

## Imports

In [1]:
import json
import re

## Experiments

### Code

In [2]:
def text_to_int_answer(text):
    try:
        answer = float(text)
        if answer < 0 or not answer.is_integer():
            return None
        return int(answer)
    except (ValueError, OverflowError):
        return None

In [3]:
def parse_boxed_answer(text):
    matches = re.findall(r'\\boxed\{(\d+)\}', text)
    if matches:
        return text_to_int_answer(matches[-1])
    return None

In [4]:
def parse_last_answer(text):
    pattern = r'(?:the answer is|the final answer is)\s*:?\s*\$?(\d+(\.\d+)?)\$?'
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        return text_to_int_answer(matches[-1][0])
    return None

def parse_answer(text):
    answer = parse_boxed_answer(text)
    if answer is not None:
        return answer
    return parse_last_answer(text)

In [5]:
test_cases = [
    ('The answer is: $651$', 651),
    ('The answer is: $5$.', 5),
    ('The answer is: 6.', 6),
    ('The final answer is 0.', 0),
    ('The final answer is 126.', 126),
    ('The final answer is: $2$.', 2),
    ('The answer is $\\boxed{3}$', 3),
    ('The answer is $\\boxed{-1}$', None),
    ('The answer is $\\boxed{1.5}$', None),
    ('The answer is: $-1$.', None),
    ('The answer is: $4.5$.', None),
    ('The final answer is 0.6', None),
]
for text, answer in test_cases:
    assert parse_answer(text) == answer, (text, parse_answer(text))
    assert parse_answer(text.lower()) == answer, text

### Verify with real results

In [6]:
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/12_prompt_variations/2024-05-18_13:46:51_results.json'
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/12_prompt_variations/2024-05-18_10:02:56_results.json'
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/12_prompt_variations/2024-05-18_10:06:31_results.json'
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/09_evaluate_MATH5/2024-05-15_11:53:30_results.json'
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/09_evaluate_MATH5/2024-05-15_11:33:15_results.json'
results_filepath = '/mnt/hdd0/Kaggle/aimo/experiments/15_prompt_engineering/2024-05-28_10:36:21_results.json'
with open(results_filepath) as f:
    data = json.load(f)

In [7]:
results = [values[0] for values in data.values()]
print(len(results))
unparsed_responses = []
for result in results:
    answer = parse_answer(result['response'])
    if answer is None:
        if not result['reached_max_tokens'] and result['coding_errors'] < 2:
            unparsed_responses.append(result)
len(unparsed_responses)

580


11

In [22]:
print(unparsed_responses[11]['response'])

IndexError: list index out of range

In [9]:
unparsed_responses[3]['response'][-100:]

'e value of $q(-1)$ is $-1$. Therefore, the answer is $\\boxed{-1}$.\nThe final answer is $\\boxed{-1}$.'

In [10]:
parse_boxed_answer(unparsed_responses[3]['response'][-100:])

In [11]:
parse_boxed_answer(unparsed_responses[3]['response'])